In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pysentiment2 as ps
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import gensim
from sklearn.feature_extraction.text import CountVectorizer
import yfinance as yf


def sentiment_vader(var):
    #pip install vaderSentiment
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
    sentiment = SentimentIntensityAnalyzer() 
    out_score = sentiment.polarity_scores(var)
    return out_score
def sentiment_ps(var):
    lm = ps.LM()
    tokens = lm.tokenize(var)
    score = lm.get_score(tokens)
    return score
def topic_distribution(string_input):
    string_input = [string_input]
    # Fit and transform
    X = vect.transform(string_input)
 
    # Convert sparse matrix to gensim corpus.
    corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)
 
    output = list(ldamodel[corpus])[0]
 
    return output
def topic_prediction(my_document):
    string_input = [my_document]
    X = vect.transform(string_input)
    # Convert sparse matrix to gensim corpus.
    corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)
    output = list(ldamodel[corpus])[0]
    topics = sorted(output,key=lambda x:x[1],reverse=True)
    return topics[0][0]
def normalize(arr):
     return (arr-min(arr))/(max(arr)-min(arr))
def normalize_2(arr):
     return np.interp(arr, (arr.min(), arr.max()), (-1, +1))

In [2]:
df = pd.read_pickle("sentiment.pkl")
df['Year']= pd.to_datetime(df['Year'])
df = df.set_index('Year')
df['Year'] = df.index

In [18]:
# Get Stock Market Data: USD/EUR exchange rate 
import pandas_datareader.data as web
stock_info = web.DataReader('EURUSD=X', 'yahoo',start= "1993-02-01", end = "2021-02-20")
stock_info

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2003-12-01,1.204007,1.194401,1.203398,1.196501,0.0,1.196501
2003-12-02,1.210903,1.194600,1.196101,1.208897,0.0,1.208897
2003-12-03,1.213003,1.207700,1.209000,1.212298,0.0,1.212298
2003-12-04,1.214403,1.204398,1.212004,1.208094,0.0,1.208094
2003-12-05,1.219096,1.206593,1.207802,1.218695,0.0,1.218695
2003-12-08,1.224005,1.215407,1.216797,1.222001,0.0,1.222001
2003-12-09,1.227702,1.219795,1.222105,1.224995,0.0,1.224995
2003-12-10,1.226603,1.216205,1.224905,1.219096,0.0,1.219096
2003-12-11,1.223496,1.212298,1.219096,1.222404,0.0,1.222404


In [19]:
df1 = df[df['Year']> "2003-12-01"]
df1

,Statement,polarity,positive,negative,subjectivity,Year
Year,,,,,,
2003-12-09,A meeting of the Federal Open Market Committee...,-0.153846,11,15,0.013039,2003-12-09
2004-01-28,A meeting of the Federal Open Market...,0.019608,26,25,0.023046,2004-01-28
2004-03-16,A meeting of the Federal Open Market Committee...,-0.205479,58,88,0.060960,2004-03-16
2004-05-04,A meeting of the Federal Open Market Committee...,-0.060241,78,88,0.070789,2004-05-04
2004-06-30,A meeting of the Federal Open Market Committee...,-0.074830,68,79,0.065333,2004-06-30
2004-08-10,A meeting of the Federal Open Market ...,-0.219178,57,89,0.069756,2004-08-10
2004-09-21,A meeting of the Federal Open Market Committee...,-0.208633,55,84,0.057249,2004-09-21
2004-11-10,A meeting of the Federal Open Market Committee...,-0.159420,58,80,0.070086,2004-11-10
2004-12-14,A meeting of the Federal Open Market Committee...,-0.149254,57,77,0.070526,2004-12-14


In [22]:
# stock market volatibility within 10 days after the Fed meetings, window = 10 performs the best
import numpy as np
price_changes = []
price_change_percentages = []
labels = []
window = 2
# workaround for rolling widow
for index, row in df1.iterrows():
    minutes_date = row['Year']
    try:
        start = stock_info.index.get_loc(minutes_date)
    except:
        start = stock_info.index.get_loc(minutes_date + pd.DateOffset(days=1))
    stock_closing_sum = stock_info.iloc[start+window]['Close']
    start_price = stock_info['Close'].iloc[start]
    price_change = stock_closing_sum - start_price
    price_change_percent = (stock_closing_sum - start_price) / start_price
    price_changes.append(price_change)
    price_change_percentages.append(price_change_percent)
    

df1['price_change'] = price_changes
df1['price_change_percent'] = price_change_percentages

#label =1 means high volatility, label = 1 means low volatility. High volatility is identified if VIX price change percent is far from its median (greater than 75% or smaller than 25%)
df1['label'] = df1['price_change_percent'].apply(lambda x: 1 if x> df1["price_change_percent"].quantile(0.75) or x < df1["price_change_percent"].quantile(0.25) else 0)

KeyError: Timestamp('2005-09-20 00:00:00')

In [ ]:
X = df1
X = np.array(X.drop(['label','Statement','Year','positive','negative','price_change','price_change_percent'],1))
y = np.array(df1['label'])

In [ ]:
#Split(80% training, 20% test)
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size= 0.4, random_state = 0)

In [ ]:
# Create and train model
model = LinearDiscriminantAnalysis().fit(x_train, y_train)

In [ ]:
predictions = model.predict(x_test)
print(classification_report(y_test,predictions))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [ ]:
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
m = randomclassifier.fit(x_train,y_train)
predictions_2 = m.predict(x_test)
score = accuracy_score(y_test,predictions)
report = classification_report(y_test,predictions)
print(score)
print(report)